# 6장. 지지 벡터 머신
## 학습 목표
 - 지지 벡터 머신 소개
 - 최적화를 위한 SMO 알고리즘
 - 데이터 '변형'을 위해 커널 사용하기
 - 다른 분류와 지지 벡터 머신 비교하기
 
## 6.1 최대 마진으로 데이터 분리하기
 - 장점: 일반화의 오류가 낮음, 계산 비용이 적음, 결과 해석이 쉬움
 - 단점: 매개변수의 조정과 커널 선택에 민감함(본래 이진 분리만을 다룸)
 - 활용: 수치형 값, 명목형 값
 <br><br>
 - 초평면 분리(separating hyperplane) : 데이터 집합을 분리하는 것 
   <br>따라서 2D 플롯에서는 하나의 선이 되지만 3차원 데이터 집합에서 데이터를 분리하기 위해서는 하나의 평면이 필요함
 - 초평면(hyperplane) : n차원 공간에서 초평면은 (n-1)차원의 집합이며 초평면은 의사 결정 범위(decision boundary)가 됨
 <br>
 <br>
 - 선형으로 분리되지 않는 데이터 집합 예제
 <img src = 'image/figure6_1.PNG' width=400 height=400>
 <br>
 - 선형 분리(linear separable)가 가능한 데이터 예제
 <img src = 'image/figure6_2.PNG' width=400 height=400>
 - B,C,D 중에서 가장 좋은 것은 D임
 > 초평면 분리에 가장 가까운 지점을 찾기를 원하면서도 이 지점이 가능한 한 분리선에서 멀리 떨어져 있기를 원함<br>
 > 마진이 가능한 한 가장 커야 함<br>
 > 실수를 하거나 한정된 데이터로 분류기를 훈련시키고자 할 때에도 분류기가 최대한 튼튼하기를 원하기 때문
 - 지지 벡터 : 초평면 분리에 가장 가까운 지점
 - 분리선에서 지지 벡터가지의 거리를 가장 크게 해야 함
 
## 6.2 최대 마진 찾기
 - 초평면 분리는 w^Tx+b로부터 얻어짐
 - 만약 점 A에서 분리면까지의 거리를 구하고자 한다면, 선에 직각 또는 수직인 선의 길이를 측정해야 함
 - 이러한 측정은 |w^Tx+b|/||w||으로 구할 수 있음
 - 상수 b는 단지 로지스틱 회귀에서 w0처럼 오프셋(offset)임
 - 여기서 w와 b는 데이터를 위한 분리선 또는 초평면을 묘사하는 것
 <img src = 'image/figure6_3.PNG' width=300 height=300>
 - 점 A에서 분리면까지의 거리는 분리면에 수직인 하나의 선에 의해 측정됨
 
### 6.2.1 분류기 관점에서의 최적화 문제 구성하기
 - 분류항목의 계산 : w^Tx+b < 0 이면 f(w^Tx+b)=-1 w^Tx+b >= 0 이면 f(w^Tx+b)=1
 - 마진의 계산 : label*(w^Tx+b)
 - 마진이 가장 작은 점을 찾으면 그 마진을 최대화해야 함 : <img src='image/maxMargin.PNG' width=300 height=150>
 - 제약 조건 : label*(w^Tx+b)이 1.0 또는 이보다 더 큰 값이 되어야 함
 - 이러한 제약적인 최적화 문제를 해결하기 위해 라그랑지 곱수(Lagrange multipliers)를 사용함 : 
 <img src='image/lagrangeMultipliers.PNG' width=300 height=150>
 위 식은 다음과 같은 제약 조건을 따름
 <img src='image/constraint.PNG' width=200 height=100>
 그러나 이것은 데이터가 선형분리된다는 가정이 필요함
 - 여유 변수(slack variable)로 의사 결정 범위에 속하지 않는 예제를 허용할 수 있음
 - 여유 변수를 도입시킨 새로운 제약 조건 : <img src='image/slackConstraint.PNG' width=200 height=100>
 상수 c는 마진을 크게 만들고자 하는 목표와 대부분의 예제가 기능적인 1.0 이상의 마진을 갖도록 보장하는 것 사이에서 가중치를 조절
 <br> 상수 c는 최적화 코드의 인자이므로 변경 가능하며 그에 따라 다른 결과를 얻을 수 있음
 - 이제 알파를 해결하면 알파의 관점에서 분리 초평면을 표현 가능
 - SVMs에서 대부분의 작업이 알파를 찾는 것임
 
### 6.2.2 일반적인 기본 구조로 지지 벡터 머신에 접근하기
 - SVMs의 일반적인 접근
 > 1. 수집: 모든 방법
 > 2. 준비: 수치형 값
 > 3. 분석: 분리 초평면을 시각화하는 데 도움이 됨
 > 4. 훈련: 대부분의 시간을 이 단계에 소비. 두 개의 매개변수는 이번 단계가 진행되는 동안 조정될 수 있음
 > 5. 검사: 매우 간단한 계산임
 > 6. 사용: 거의 모든 분류 문제에 SVMs를 사용할 수 있으나 SVMs는 이진 분류기 이므로 3개 이상의 분류항목인 경우 수정이 필요함
 
## 6.3 SMO 알고리즘으로 효율적인 최적화하기
 - 이차 방정식 솔버(quadratic solver): 선형적인 제약 조건이 있는 여러 변수를 가지고 이차 방정식 함수를 최적화하는 소프트웨어의 한 부분
### 6.3.1 플랫의 SMO 알고리즘
 - 1996년 존 플랫(John Platt)이 지지 벡터 머신을 훈련시키기 위해 SMO라는 알고리즘을 발표함
 - SMO는 순차적 최소 최적화(Sequential Minimal Optimization)의 약어
 - SMO 알고리즘은 알파와 b의 집합을 찾는 것 -> 알파의 집합을 구하게 되면 가중치 w는 쉽게 계산되며 분리 초평면을 구할 수 있음
 - SMO 알고리즘의 동작 :
  > 각각의 사이클을 최적화하기 위해 두 개의 알파를 선택
  > 적당한 알파의 쌍을 찾게 되면, 그 중 하나는 값을 증가시키고 하나는 줄임
  > 적당한 알파를 구하기 위해서는 알파 집합이 조건을 정확하게 충족해야 함: 
      > - 알파 쌍 모두가 그들의 마진 경계 밖에 있어야 함
      > - 알파가 이미 고정되어 있거나 경계를 갖지 않아야 함

### 6.3.2 간략한 형태의 SMO로 적은 양의 데이터 집합 해결하기

In [61]:
# 6.1 SMO 알고리즘을 위한 도움 함수

def loadDataSet(fileName):
    dataMat=[]; labelMat=[]
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

def selectJrand(i, m):
    j=i
    while(j==i):
        j=int(random.uniform(0,m))
    return j

def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

In [62]:
dataArr, labelArr = loadDataSet('data/testSEt.txt')
print(labelArr)

[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


In [63]:
# 6.2 간략한 형태의 SMO 알고리즘
from numpy import *

def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = mat(dataMatIn); labelMat = mat(classLabels).transpose()
    b=0; m,n=shape(dataMatrix)
    alphas = mat(zeros((m,1)))
    iter = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m):
            fXi=float(multiply(alphas, labelMat).T * (dataMatrix*dataMatrix[i, :].T)) + b
            Ei = fXi - float(labelMat[i])
            if((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i]>0)):
                j = selectJrand(i, m)
                fXj = float(multiply(alphas, labelMat).T * (dataMatrix*dataMatrix[j, :].T))+b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                if(labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: print("L==H"); continue
                eta = 2.0 * dataMatrix[i, :] * dataMatrix[j, :].T \
                        - dataMatrix[i, :] * dataMatrix[i, :].T \
                        - dataMatrix[j, :] * dataMatrix[j, :].T
                if eta >= 0: print("eta>=0"); continue
                alphas[j] -= labelMat[j]*(Ei - Ej) / eta
                alphas[j] = clipAlpha(alphas[j], H, L)
                if(abs(alphas[j] - alphaJold) < 0.00001): print("j not moving enough"); continue
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*\
                    dataMatrix[i,:]*dataMatrix[i,:].T - \
                    labelMat[j]*(alphas[j]-alphaJold)*\
                    dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*\
                    dataMatrix[i,:]*dataMatrix[j,:].T - \
                    labelMat[j]*(alphas[j]-alphaJold)*\
                    dataMatrix[j,:]*dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): 
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1 + b2) / 2.0
                alphaPairsChanged += 1
                print ("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
        if (alphaPairsChanged == 0): 
            iter += 1
        else:
            iter = 0
        print("iteration number: %d" % iter)
    return b, alphas                

In [64]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

iter: 0 i:0, pairs changed 1
iter: 0 i:2, pairs changed 2
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:23, pairs changed 3
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:39, pairs changed 4
L==H
j not moving enough
iter: 0 i:48, pairs changed 5
iter: 0 i:52, pairs changed 6
L==H
iter: 0 i:55, pairs changed 7
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
L==H
j not moving enough
iter: 0 i:39, pairs changed 1
L==H
L==H
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
L==H
iteration number: 0
j not moving enough
j not moving enough
L==H
j not moving enough
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving

In [65]:
print(b)

[[-3.81058178]]


In [66]:
print(alphas[alphas>0])

[[ 0.04041764  0.31873109  0.10974026  0.24940847]]


In [67]:
print(shape(alphas[alphas>0]))
for i in range(100):
    if alphas[i]>0.0:
        print(dataArr[i], labelArr[i])

(1, 4)
[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[5.286862, -2.358286] 1.0
[6.080573, 0.418886] 1.0


## 6.4 전체 플랫 SMO를 이용해 최적화 속도 올리기

In [68]:
# 6.3 전체 플랫 SMO에 대한 지지 함수

class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2)))
        
def calcEk(oS, k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T*\
                (oS.X*oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

def selectJ(i, oS, Ei):
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:
            if k == i: continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1,Ek]

In [69]:
# 6.4 전체 플랫 SMO 최적화 절차

def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or\
        ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
            j,Ej = selectJ(i, oS, Ei)
            alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
            if (oS.labelMat[i] != oS.labelMat[j]):
                L = max(0, oS.alphas[j] - oS.alphas[i])
                H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
            else:
                L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
                H = min(oS.C, oS.alphas[j] + oS.alphas[i])
            if L==H: print("L==H"); return 0
            eta = 2.0 * oS.X[i,:]*oS.X[j,:].T - oS.X[i,:]*oS.X[i,:].T - \
                oS.X[j,:]*oS.X[j,:].T
            if eta >= 0: print("eta>=0"); return 0
            oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
            oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
            updateEk(oS, j)
            if (abs(oS.alphas[j] - alphaJold) < 0.00001):
                print("j not moving enough"); return 0
            oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*\
                        (alphaJold - oS.alphas[j])
            updateEk(oS, i)
            b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*\
                oS.X[i,:]*oS.X[i,:].T - oS.labelMat[j]*\
                (oS.alphas[j]-alphaJold)*oS.X[i,:]*oS.X[j,:].T
            b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*\
                oS.X[i,:]*oS.X[j,:].T - oS.labelMat[j]*\
                (oS.alphas[j]-alphaJold)*oS.X[j,:]*oS.X[j,:].T
            if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
            elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
            else: oS.b = (b1 + b2)/2.0
            return 1
    else: return 0

In [34]:
# 6.5 전체 플랫 SMO 외부 반복문

def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler) 
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL(i,oS)
            print ("fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        else:
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print ("non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        if entireSet: entireSet = False
        elif (alphaPairsChanged == 0): entireSet = True
        print ("iteration number: %d" % iter)
    return oS.b,oS.alphas

In [35]:
dataArr, labelArr = loadDataSet('data/testSet.txt')
b, alphas = smoP(dataArr, labelArr, 0.6, 0.001, 40)

L==H
L==H
L==H
j not moving enough
L==H
L==H
L==H
L==H
L==H
L==H
j not moving enough
L==H
L==H
j not moving enough
L==H
L==H
L==H
L==H
L==H
j not moving enough
fullSet, iter: 0 i:99, pairs changed 6
iteration number: 1
j not moving enough
non-bound, iter: 1 i:0, pairs changed 0
j not moving enough
non-bound, iter: 1 i:3, pairs changed 0
j not moving enough
non-bound, iter: 1 i:4, pairs changed 0
j not moving enough
non-bound, iter: 1 i:17, pairs changed 0
j not moving enough
non-bound, iter: 1 i:18, pairs changed 0
j not moving enough
non-bound, iter: 1 i:25, pairs changed 0
j not moving enough
non-bound, iter: 1 i:46, pairs changed 0
non-bound, iter: 1 i:55, pairs changed 0
non-bound, iter: 1 i:94, pairs changed 0
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
L==H
j not moving en

## 6.5 더 복잡한 데이터를 위해 커널 사용하기
### 6.5.2 반지름 성향 함수
 - 반지름 성향 함수(Radial Bias function)은 커널이며, 종종 지지 벡터 머신과 함께 사용됨
 - 반지름 성향 함수는 하나의 벡터를 가지고 벡터간의 거리를 기반으로 하는 하나의 스칼라를 출력하는 함수
 - 가우스 확률 분포를 사용 : <img src='image/gaussian.PNG' width=200 height=100>

In [46]:
# 6.6 커널 변환 함수

def kernelTrans(X, A, kTup):
    m,n = shape(X)
    K = mat(zeros((m,1)))
    if kTup[0]=='lin': K = X * A.T
    elif kTup[0]=='rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow*deltaRow.T
        K = exp(K /(-1*kTup[1]**2))
    else: raise NameError('Houston We Have a Problem -- \
    That Kernel is not recognized')
    return K

class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2)))
        self.K = mat(zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)

In [47]:
# 6.7 커널을 사용하기 위해 필요한 함수 innerL()과 calcEk()의 변경
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or\
        ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
            j,Ej = selectJ(i, oS, Ei)
            alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
            if (oS.labelMat[i] != oS.labelMat[j]):
                L = max(0, oS.alphas[j] - oS.alphas[i])
                H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
            else:
                L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
                H = min(oS.C, oS.alphas[j] + oS.alphas[i])
            if L==H: print("L==H"); return 0
            eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j]
            if eta >= 0: print("eta>=0"); return 0
            oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
            oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
            updateEk(oS, j)
            if (abs(oS.alphas[j] - alphaJold) < 0.00001):
                print("j not moving enough"); return 0
            oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*\
                        (alphaJold - oS.alphas[j])
            updateEk(oS, i)
            b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] -\
                oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
            b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]-\
                oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
            if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
            elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
            else: oS.b = (b1 + b2)/2.0
            return 1
    else: return 0
    
def calcEk(oS, k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

### 6.5.3 검사를 위한 커널 사용

In [53]:
# 6.8 커널으로 분류하기 위한 반지름 성향 검사 함수
from numpy import *
def testRbf(k1=1.3):
    dataArr, labelArr = loadDataSet('data/testSetRBF.txt')
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1))
    dataMat = mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd = nonzero(alphas.A>0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]
    print("there are %d Support Vector" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kenelEval = kernelTrans(sVs, datMat[i,:],('rbf', k1))
        predict=kernelEval.T * multiply(labelSV, alphs[svInd]) + b
        if sign(predict)!=sign(labelArr[i]):
            errorCount += 1
    print("the training error rate is: %f" % (float(errorCount)/m))
    dataArr, labelArr = loadDataSet('data/testSetRBF2.txt')
    errorCount=0
    datMat=mat(dataArr); labelMat=mat(labelArr).transpose()
    m,n=shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i,:],('rbf',k1))
        predict = kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]):
            errorCount += 1
    print("the test error rate is: %f" % (float(errorCount)/m))       

In [54]:
svmMLiA.testRbf()

L==H
fullSet, iter: 0 i:0, pairs changed 0
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 3
fullSet, iter: 0 i:5, pairs changed 4
fullSet, iter: 0 i:6, pairs changed 4
fullSet, iter: 0 i:7, pairs changed 5
fullSet, iter: 0 i:8, pairs changed 5
fullSet, iter: 0 i:9, pairs changed 5
fullSet, iter: 0 i:10, pairs changed 6
fullSet, iter: 0 i:11, pairs changed 7
fullSet, iter: 0 i:12, pairs changed 7
fullSet, iter: 0 i:13, pairs changed 8
fullSet, iter: 0 i:14, pairs changed 9
fullSet, iter: 0 i:15, pairs changed 10
fullSet, iter: 0 i:16, pairs changed 11
fullSet, iter: 0 i:17, pairs changed 12
fullSet, iter: 0 i:18, pairs changed 13
fullSet, iter: 0 i:19, pairs changed 14
fullSet, iter: 0 i:20, pairs changed 14
fullSet, iter: 0 i:21, pairs changed 15
fullSet, iter: 0 i:22, pairs changed 15
fullSet, iter: 0 i:23, pairs changed 16
fullSet, iter: 0 i:24, pairs changed 17
j not moving enough


## 6.6 예제: 필기체 인식 예제 다시 적용하기
 - 예제: SVMs로 숫자 인식하기
 > 1. 수집: 제공된 텍스트 파일
 > 2. 준비: 이진형 이미지로부터 벡터를 생성함
 > 3. 분석: 시각적으로 이미지 벡터를 검토
 > 4. 훈련: 두 가지 다른 커널과 반지름 성향 커널에 대한 다른 설정으로 SMO 알고리즘을 실행
 > 5. 검사: 다른 커널을 검사하기 위한 함수를 작성하고 오류율을 계산
 > 6. 사용: 이미지 인식에 대한 전체 응용 프로그램은 여기서 다루지 않는 몇 가지 이미지 처리를 요구함

In [57]:
# 6.9 지지 벡터 머신 필기체 인식

def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9: hwLabels.append(-1)
        else: hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels

def testDigits(kTup=('rbf', 10)):
    dataArr,labelArr = loadImages('trainingDigits')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd]
    labelSV = labelMat[svInd];
    print ("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print ("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadImages('testDigits')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print ("the test error rate is: %f" % (float(errorCount)/m))

In [59]:
svmMLiA.testDigits(('rbf',20))

L==H
fullSet, iter: 0 i:0, pairs changed 0
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 3
fullSet, iter: 0 i:5, pairs changed 4
fullSet, iter: 0 i:6, pairs changed 5
fullSet, iter: 0 i:7, pairs changed 6
fullSet, iter: 0 i:8, pairs changed 6
fullSet, iter: 0 i:9, pairs changed 6
fullSet, iter: 0 i:10, pairs changed 6
fullSet, iter: 0 i:11, pairs changed 6
fullSet, iter: 0 i:12, pairs changed 7
fullSet, iter: 0 i:13, pairs changed 8
fullSet, iter: 0 i:14, pairs changed 9
j not moving enough
fullSet, iter: 0 i:15, pairs changed 9
j not moving enough
fullSet, iter: 0 i:16, pairs changed 9
j not moving enough
fullSet, iter: 0 i:17, pairs changed 9
j not moving enough
fullSet, iter: 0 i:18, pairs changed 9
fullSet, iter: 0 i:19, pairs changed 10
fullSet, iter: 0 i:20, pairs changed 11
fullSet, iter: 0 i:21, pairs changed 12
L==H
fullSet, iter: 0 i:22, pairs changed 12
L==H
fullSet, iter: